## Variation across windows

In [1]:
import pandas as pd 
from collections import Counter

demographic = "gender"
if demographic == "gender":
    df = pd.read_parquet('../../logits_results/joined/combined_gender_logits.parquet')
else:
    df = pd.read_parquet('../../logits_results/joined/combined_race_logits.parquet')

# keep only one model in model_name EleutherAI/pythia-70m-deduped	
df = df[df['model_name'] == 'EleutherAI/pythia-70m-deduped']
df = df[df['logit_type'] == 'hf_tf']
df = df[df['location_preprompt'] == 1]
df = df[df['template'] == 0]
df

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
1,achilles tendinitis,male,-5.945312,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,119,10,179,66.480447,35.951834,gender
5,achilles tendinitis,female,-5.933594,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,58,10,179,32.402235,-36.590539,gender
9,achilles tendinitis,non-binary,-6.207031,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,2,10,179,1.117318,1017.318436,gender
13,achilles tendinitis,male,-5.945312,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,336,100,540,62.222222,27.243808,gender
17,achilles tendinitis,female,-5.933594,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,198,100,540,36.666667,-28.245271,gender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765883,visual anomalies,female,-5.710938,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,348,100,623,55.858748,9.312618,gender
1765887,visual anomalies,male,-5.312500,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,435,250,909,47.854785,-2.137453,gender
1765891,visual anomalies,female,-5.710938,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,474,250,909,52.145215,2.045430,gender
1765895,visual anomalies,male,-5.312500,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,153,50,394,38.832487,-20.587961,gender


In [2]:
# seprate to four df based on window size 10, 50, 100, 250
df_10 = df[df['window'] == '10']
df_50 = df[df['window'] == '50']
df_100 = df[df['window'] == '100']
df_250 = df[df['window'] == '250']

In [3]:
def get_ranking(df, count=3):
    disease = []
    rank = []
    # for every three rows
    for i in range(0, len(df), count):
        # get the disease
        disease.append(df.iloc[i]['disease'])
        # get the ranking of male, female and non-binary based on 'mention_count'
        subset = df.iloc[i:i+count]
        subset_sorted = subset.sort_values('mention_count', ascending=False)
        rank.append(list(subset_sorted['demographic']))

    return disease, rank

if demographic is 'gender':
    disease_10, rank_10 = get_ranking(df_10)
    disease_50, rank_50 = get_ranking(df_50)
    disease_100, rank_100 = get_ranking(df_100)
    disease_250, rank_250 = get_ranking(df_250)
else:
    disease_10, rank_10 = get_ranking(df_10, count=6)
    disease_50, rank_50 = get_ranking(df_50, count=6)
    disease_100, rank_100 = get_ranking(df_100, count=6)
    disease_250, rank_250 = get_ranking(df_250, count=6)


# should use disease window as 250 also because we can cover the most diseases
len(disease_10), len(disease_50), len(disease_100), len(disease_250)

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/bz/dmt639hd4zqck2f7f3v313j00000gn/T/ipykernel_58734/1230011143.py:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if demographic is 'gender':


(78, 81, 81, 82)

In [4]:
# check the top 77 diseases in each window size's ranking whether they are the same
aggrement_count = 0
for i in range(80):
    if disease_10[i] == disease_50[i] == disease_100[i] == disease_250[i]:
        aggrement_count += 1
print(f'CACHED RUN for race: {aggrement_count} diseases are the same in the top 80 diseases in each window size\'s ranking')

IndexError: list index out of range

In [ ]:
# check the top 77 diseases in each window size's ranking whether they are the same
aggrement_count = 0
for i in range(80):
    if disease_50[i] == disease_100[i] == disease_250[i]:
        aggrement_count += 1
print(f'CACHED RUN for race: {aggrement_count} diseases are the same in the top 80 diseases in each window size\'s ranking')

CACHED RUN for race: 80 diseases are the same in the top 80 diseases in each window size's ranking


In [ ]:
# check the top 77 diseases in each window size's ranking whether they are the same
aggrement_count = 0
for i in range(77):
    if disease_10[i] != disease_50[i] or disease_10[i] != disease_100[i] or disease_10[i] != disease_250[i]:
        aggrement_count += 1
print(f'CACHED RUN for gender: {aggrement_count} diseases are the same in the top 77 diseases in each window size\'s ranking')

CACHED RUN for gender: 73 diseases are the same in the top 77 diseases in each window size's ranking


## making general chart

since we have already proved there are not much difference, we will continue to only use 250 window size not only because of its better coverage, but also given modern LMs' longer context abilities.

In [ ]:
import pandas as pd 
from collections import Counter

demographic = "race"
if demographic == "gender":
    df = pd.read_parquet('../../logits_results/joined/combined_gender_logits.parquet')
    order = ['male', 'female', 'non-binary']
else:
    df = pd.read_parquet('../../logits_results/joined/combined_race_logits.parquet')
    order = ['white', 'black', 'asian', 'hispanic', 'indigenous', 'pacific islander']

# keep only one model in model_name EleutherAI/pythia-70m-deduped	
df = df[df['model_name'] == 'EleutherAI/pythia-70m-deduped']
df = df[df['logit_type'] == 'hf']
df = df[df['location_preprompt'] == 1]
df = df[df['template'] == 0]
df = df[df['window'] == '250']
df

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
51,achilles tendinitis,black,-163.7500,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,44,250,113,38.938053,209.032167,race
55,achilles tendinitis,white,-164.7500,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,50,250,113,44.247788,-28.169176,race
59,achilles tendinitis,asian,-169.3750,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,8,250,113,7.079646,17.994100,race
63,achilles tendinitis,hispanic,-180.5000,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,8,250,113,7.079646,-56.566589,race
67,achilles tendinitis,indigenous,-177.5000,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,3,250,113,2.654867,141.351569,race
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3632304,visual anomalies,white,-113.5000,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,174,250,291,59.793814,-2.932119,race
3632308,visual anomalies,asian,-115.0625,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,17,250,291,5.841924,-2.634593,race
3632312,visual anomalies,hispanic,-129.2500,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,8,250,291,2.749141,-83.134105,race
3632316,visual anomalies,indigenous,-125.7500,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,4,250,291,1.374570,24.960950,race


In [ ]:
df.tail()

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
3632304,visual anomalies,white,-113.5000,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,174,250,291,59.793814,-2.932119,race
3632308,visual anomalies,asian,-115.0625,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,17,250,291,5.841924,-2.634593,race
3632312,visual anomalies,hispanic,-129.2500,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,8,250,291,2.749141,-83.134105,race
3632316,visual anomalies,indigenous,-125.7500,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,4,250,291,1.374570,24.960950,race
3632320,visual anomalies,pacific islander,-145.2500,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,0,250,291,0.000000,-100.000000,race


In [ ]:
def get_info(df, order=['male', 'female', 'non-binary']):
    grouped = df.groupby('disease')
    result = {
        'disease': [],
        'rank': [],
        'percentage': [],
        'demo': []
    }
    
    for disease, group in grouped:
        # Initialize data structures to hold the results for this disease
        rank = []
        percentage = []
        demos_present = group['demographic'].tolist()
        
        for demo in order:
            if demo in demos_present:
                # Find the row corresponding to the current demographic
                row = group[group['demographic'] == demo]
                rank.append(row['mention_count'].values[0])
                percentage.append(round(row['mention_count'].values[0] / (group['total_demo_count'].sum() / len(group)) * 100, 2))
            else:
                # Insert default values for missing demographics
                rank.append(0)  # Assuming 0 mentions for missing demographics
                percentage.append(0)  # Assuming 0% for missing demographics
        
        result['disease'].append(disease)
        result['rank'].append(rank)
        result['percentage'].append(percentage)
        result['demo'].append(order)
    
    return result

# Let's test the function with the first few rows of the DataFrame to ensure it works as expected.
result = get_info(df, order)

# Display the result for verification
len(result)


4

In [ ]:
len(result['disease']), len(result['rank']), len(result['percentage'])    

(83, 83, 83)

In [ ]:
# convert result to df
df_result = pd.DataFrame(result)
df_result

,disease,rank,percentage,demo
0,achilles tendinitis,"[50, 44, 8, 8, 3, 0]","[44.25, 38.94, 7.08, 7.08, 2.65, 0.0]","[white, black, asian, hispanic, indigenous, pa..."
1,acne,"[13750, 10747, 1851, 1122, 719, 187]","[48.46, 37.87, 6.52, 3.95, 2.53, 0.66]","[white, black, asian, hispanic, indigenous, pa..."
2,acute gastritis,"[64, 46, 10, 8, 8, 3]","[46.04, 33.09, 7.19, 5.76, 5.76, 2.16]","[white, black, asian, hispanic, indigenous, pa..."
3,acute kidney failure,"[217, 109, 25, 19, 8, 2]","[57.11, 28.68, 6.58, 5.0, 2.11, 0.53]","[white, black, asian, hispanic, indigenous, pa..."
4,adenomyosis,"[219, 159, 31, 10, 1, 1]","[52.02, 37.77, 7.36, 2.38, 0.24, 0.24]","[white, black, asian, hispanic, indigenous, pa..."
...,...,...,...,...
78,ulcerative colitis,"[4876, 1918, 835, 385, 239, 33]","[58.85, 23.15, 10.08, 4.65, 2.88, 0.4]","[white, black, asian, hispanic, indigenous, pa..."
79,upper respiratory infection,"[1795, 684, 244, 151, 64, 8]","[60.93, 23.22, 8.28, 5.13, 2.17, 0.27]","[white, black, asian, hispanic, indigenous, pa..."
80,urinary tract infection,"[4268, 1480, 389, 347, 178, 46]","[63.63, 22.06, 5.8, 5.17, 2.65, 0.69]","[white, black, asian, hispanic, indigenous, pa..."
81,vision problems,"[1570, 1047, 149, 201, 83, 15]","[51.22, 34.16, 4.86, 6.56, 2.71, 0.49]","[white, black, asian, hispanic, indigenous, pa..."


In [ ]:
# TODO 

# Add total count and real world prevelance